In [1]:
import logging
import json
import requests
import numpy as np
import pandas as pd
from shapely.geometry import LineString, Polygon, Point
from folium import plugins
import folium
import numpy
import matplotlib.pyplot as plt
import utm

OPTIMIZED_ROUTE_URL = "http://localhost:8002/optimized_route" 
DEFAULT_HEADERS = {"Content-type": "application/json"}

# Use six degrees of precision when using Valhalla for routing
VALHALLA_PRECISION = 1.0 / 1e6

EARTH_RADIUS = 6371. * 1000. # In meters
 
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

In [2]:
class DataPoint:
    
    def __init__(self, latitude, longitude):
        self.latitude = latitude
        self.longitude = longitude
        

class ValhallaInterface:

    def __init__(self) -> None:
        pass


    def decode_polyline(self, polyline_string):
        index = 0; latitude = 0; longitude = 0
        coordinates = []
        changes = {"latitude": 0, "longitude": 0}
        # Coordinates have variable length when encoded, so just keep
        # track of whether we have hit the end of the string. In each
        # while loop iteration a single coordinate is decoded.
        while index < len(polyline_string):
            # Gather latitude/longitufe changes, store them in a dictionary to apply them later
            for unit in ["latitude", "longitude"]: 
                shift, result = 0, 0
                while True:
                    byte = ord(polyline_string[index]) - 63
                    index += 1
                    result |= (byte & 0x1f) << shift
                    shift += 5
                    if not byte >= 0x20:
                        break
                if (result & 1):
                    changes[unit] = ~(result >> 1)
                else:
                    changes[unit] = (result >> 1)
            latitude += changes["latitude"]
            longitude += changes["longitude"]
            coordinates.append(
                [VALHALLA_PRECISION * latitude, VALHALLA_PRECISION * longitude],
            )
        return coordinates

    def send_optimized_route_request(self, dp1, dp2):    
        def build_optimized_route_request(dp1, dp2):
            return json.dumps({
                "locations":[
                    # Start location
                    {"lat": dp1.latitude, "lon": dp1.longitude},
                    # End location
                    {"lat": dp2.latitude, "lon": dp2.longitude},
                ],
                "costing": "pedestrian",
                "directions_options": {
                    "units":"kilometers"
                },
            })
        d = build_optimized_route_request(dp1, dp2)
        response = requests.post(
            OPTIMIZED_ROUTE_URL,
            data = d,
            headers = DEFAULT_HEADERS,
        )
        if response.status_code == 200:
            content = json.loads(response.content)
        else:
            content = None
        return content

def setup_map(center, zoom_start = 14, tiles: str = "cartodbdark_matter"):
    map_ = folium.Map(
        location = center,
        zoom_start = zoom_start,
#         tiles = tiles,
    )
    plugins.Fullscreen(
        position = "topleft"
    ).add_to(map_)
    plugins.Draw(
        filename="placeholder.geojson",
        export = True,
        position = "topleft"
    ).add_to(map_)
    return map_

def plot_point(datapoint, center, color, radius = 5.0, opacity = 1, map_ = None):
    folium.CircleMarker(
        [datapoint.latitude, datapoint.longitude],
        radius = radius,
        color = color,
        opacity = opacity,
        popup = f"...",
    ).add_to(map_)
    return map_
    
def plot_polyline(datapoints, center, color, weight: float = 2.0, opacity: float = 1, map_ = None):
        lst = []
        for datapoint in datapoints:
            lst.append([
                datapoint.latitude,
                datapoint.longitude,
            ])
        folium.PolyLine(
            lst,
            color = color,
            weight = weight,
            opacity = opacity,
            popup = f"...",
        ).add_to(map_)
        return map_

def test_valhalla(start_dp, end_dp):
    d = json.dumps({
        "locations":[
            {"lat": start_dp.latitude, "lon": start_dp.longitude},
            {"lat": end_dp.latitude, "lon": end_dp.longitude},
#             {"lat": 55.39594, "lon": 10.38831},
#             {"lat": 55.39500, "lon": 10.38800},
        ],
        "costing": "pedestrian",
        "directions_options": {
            "units":"kilometers"
        },
    })
    response = requests.post(
        OPTIMIZED_ROUTE_URL,
        data = d,
        headers = DEFAULT_HEADERS,
    )
    return response

def haversine_distance(
        lat_1: float, lon_1: float,
        lat_2: float, lon_2: float,
    ) -> float:
    """Calculate the 'Haversine' (great-circle) distance in meters between two locations
    / (latitude, longitude) points.

    Args:
        lat_1 (float): Latitude of location 1.
        lon_1 (float): Longitude of location 1.
        lat_2 (float): Latitude of location 2.
        lon_2 (float): Longitude of location 2.

    Returns:
        float: The distance between location 1 and 2 in meters. 
    """
    d_lat = np.radians(lat_1 - lat_2)
    d_lon = np.radians(lon_1 - lon_2)
    lat1 = np.radians(lat_1)
    lat2 = np.radians(lat_2)
    a = np.sin(d_lat / 2) * np.sin(d_lat / 2) + \
        np.sin(d_lon / 2) * np.sin(d_lon / 2) * np.cos(lat1) * np.cos(lat2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    d = EARTH_RADIUS * c
    return d

In [3]:
# 55.363451206145925, 10.412869042981713
# start_dp = DataPoint(latitude = 55.39594, longitude = 10.38831)
start_dp = DataPoint(latitude = 55.363451206145925, longitude = 10.412869042981713)
# 55.373384555955425, 10.40937928427195
# end_dp = DataPoint(latitude = 55.39500, longitude = 10.38800)
end_dp = DataPoint(latitude = 55.373384555955425, longitude = 10.40937928427195)

response = test_valhalla(start_dp = start_dp, end_dp = end_dp)
dict_ = json.loads(response.content)
for key in dict_:
    print("Ket: ", key)
    for item in dict_[key]:
        print(item, dict_[key][item])
        print()


Ket:  trip
locations [{'type': 'break', 'lat': 55.363451, 'lon': 10.412869, 'city': 'right', 'original_index': 0}, {'type': 'break', 'lat': 55.373384, 'lon': 10.409379, 'original_index': 1}]

legs [{'maneuvers': [{'type': 2, 'instruction': 'Walk south on Munkebjergvej.', 'verbal_pre_transition_instruction': 'Walk south on Munkebjergvej.', 'verbal_post_transition_instruction': 'Continue for 800 meters.', 'street_names': ['Munkebjergvej'], 'time': 541.927, 'length': 0.767, 'cost': 541.927, 'begin_shape_index': 0, 'end_shape_index': 18, 'travel_mode': 'pedestrian', 'travel_type': 'foot'}, {'type': 10, 'instruction': 'Turn right onto Hestehaven.', 'verbal_transition_alert_instruction': 'Turn right onto Hestehaven.', 'verbal_pre_transition_instruction': 'Turn right onto Hestehaven.', 'verbal_post_transition_instruction': 'Continue for 600 meters.', 'street_names': ['Hestehaven'], 'time': 434.823, 'length': 0.615, 'cost': 439.823, 'begin_shape_index': 18, 'end_shape_index': 34, 'travel_mode'

In [44]:
vi = ValhallaInterface()
polyline = vi.decode_polyline(dict_["trip"]["legs"][0]["shape"])
polyline

[[55.363464, 10.412949],
 [55.363153, 10.413096999999999],
 [55.362933, 10.413195],
 [55.362159, 10.413471],
 [55.362018, 10.413521],
 [55.361368, 10.413751999999999],
 [55.360913, 10.4139],
 [55.359846, 10.414159999999999],
 [55.359296, 10.414391],
 [55.358928999999996, 10.414605],
 [55.358444999999996, 10.414982],
 [55.357896, 10.415536],
 [55.357647, 10.415835],
 [55.357510999999995, 10.41602],
 [55.357352, 10.416241999999999],
 [55.357234999999996, 10.416419999999999],
 [55.357127, 10.416596],
 [55.357094999999994, 10.416649999999999],
 [55.357037999999996, 10.416743],
 [55.356950999999995, 10.416604],
 [55.356843999999995, 10.416489],
 [55.356795999999996, 10.416324999999999],
 [55.356761999999996, 10.416115999999999],
 [55.356759999999994, 10.415766],
 [55.356769, 10.414829],
 [55.356786, 10.414240999999999],
 [55.356832999999995, 10.413103999999999],
 [55.356856, 10.412588],
 [55.356927, 10.410877],
 [55.356975, 10.409846],
 [55.356983, 10.408877],
 [55.356958999999996, 10.40825

In [43]:
center = [55.39594, 10.38831]

map_ = setup_map(center)
map_ = plot_point(start_dp, center, color = "red", map_ = map_)
map_ = plot_point(end_dp, center, color = "blue", map_ = map_)
map_ = plot_polyline(
    [start_dp] + [DataPoint(x, y) for x, y in polyline] + [end_dp],
    center,
    color = "blue",
    map_ = map_,
)
map_

In [45]:
# np.random.seed(20)
def sample_path_batch(M, N):
    dt = 1.0 / (N - 1)
    dt_sqrt = numpy.sqrt(dt)
    B = numpy.empty((M, N), dtype=numpy.float32)
    B[:, 0] = 0
    for n in range(N - 2):                                           
        t = n * dt
        xi = numpy.random.randn(M) * dt_sqrt
        B[:, n + 1] = B[:, n] * (1 - dt / (1 - t)) + xi
    B[:, -1] = 0
    return B

# nvals = sample_path_batch(1, 10 + 1)[0]
# ovals = [i + 10 for i in range(2)]
# uvals = [i * 10 for i in range(len(ovals))]

# # using the variable axs for multiple Axes
# fig, axs = plt.subplots(3, 1, sharex = True, sharey = True)

# axs[0].scatter(uvals, ovals)
# axs[1].scatter([i for i in range(len(nvals))], nvals)

# avals = []
# for i in range(2, len(ovals) + 1):
#     vvals = ovals[i - 2:i]
# #     min_lon = np.min(ovals[i - 2:i])
# #     max_lon = np.max(ovals[i - 2:i])
# #     min_lat = np.min(ovals[i - 2:i])
# #     max_lat = np.max(ovals[i - 2:i])
#     for j in range(len(nvals)):
#         j_val = nvals[j]
#         # min_val = np.min(nvals)
#         # max_val = np.max(nvals)
#         nval = j_val + (vvals[0] *(uvals[1] - j) + vvals[1] * (j - uvals[0])) / (uvals[1] - uvals[0])
#         # nval = min_lon + ((j_val - min_val) * (max_lon - min_lon)) / (max_val - min_val)
#         avals.append(nval)

# axs[2].scatter([i for i in range(len(avals))], avals)

# # plt.scatter([i for i in range(len(values))], values)
# # plt.plot([i for i in range(len(values))], values)
# plt.show()


In [100]:
# lat = [x for x, y in polyline]
# lon = [y for x, y in polyline]

lat = []
lon = []
for pair in polyline:    
    x, y, _, _ = utm.from_latlon(pair[0], pair[1])
    lat.append(x)
    lon.append(y)

def interpolate(linestring: LineString, distance_delta: float) -> LineString:
    n = int(linestring.length // distance_delta) + 1
    points = [
        linestring.interpolate(i / float(n - 1), normalized = True) for i in range(n)
    ]
    points_ = []
    for point in points:
#         x, y = utm.to_latlon(point.x, point.y, 32, "U")
        rand = np.random.normal(0, 1, size = 2)
#         print([x + rand[0], y + rand[1]])
        points_.append(
            [point.x + rand[0], point.y + rand[1]]
        )
#         points_.append(
#             [point.x, point.y]
#         )
#         points_.append(
#             [x, y]
#         )
    return LineString(points_)
    

In [101]:

ls = LineString(zip(lat, lon))
npolyline = interpolate(ls, 250)
npolyline.length

lat = []
lon = []
for x, y in list(npolyline.coords):
    lat.append(x)
    lon.append(y)

# map_ = setup_map(center)
# map_ = plot_point(start_dp, center, color = "red", map_ = map_)
# map_ = plot_point(end_dp, center, color = "blue", map_ = map_)
# map_ = plot_polyline(
#     [start_dp] + [DataPoint(*utm.to_latlon(x, y, 32, "U")) for x, y in list(npolyline.coords)] + [end_dp],
# #     [start_dp] + [DataPoint(x, y) for x, y in list(npolyline.coords)] + [end_dp],
#     center,
#     color = "blue",
#     map_ = map_,
# )
# map_


In [9]:
# lon_nvals = []
# lat_nvals = []
# dist = 10.
# for i in range(2, len(polyline) + 1):
#     min_lon = np.min(lon[i - 2:i])
#     max_lon = np.max(lon[i - 2:i])
#     min_lat = np.min(lat[i - 2:i])
#     max_lat = np.max(lat[i - 2:i])
#     # NOTE: Use one brownian bridge at the moment
#     values = sample_path_batch(1, 3)[0]
#     for j in range(len(values)):
#         j_val = values[j]
#         min_val = np.min(values)
#         max_val = np.max(values)
#         nval = min_lon + ((j_val - min_val) * (max_lon - min_lon)) / (max_val - min_val)
#         if j != 0 and j != len(values):
#             lon_nvals.append(nval)
#     values = sample_path_batch(1, 3)[0]
#     for j in range(len(values)):
#         j_val = values[j]
#         min_val = np.min(values)
#         max_val = np.max(values)
#         nval = min_lat + ((j_val - min_val) * (max_lat - min_lat)) / (max_val - min_val)
#         if j != 0 and j != len(values):
#             lat_nvals.append(nval)
#     if i == 1:
#         break

# coords = zip(lat_nvals, lon_nvals)
# # haversine_distance()

# #     print(min_lon - min_lon / (max_lon - min_lon))
# #     print(max_lon - min_lon / (max_lon - min_lon))
# #     print(lon[i - 2:i])

In [104]:
lon_nvals = []
lat_nvals = []
param = 100
# for i in range(2, len(polyline) + 1):
for i in range(2, len(list(npolyline.coords)) + 1):
    min_lon = np.min(lon[i - 2:i])
    max_lon = np.max(lon[i - 2:i])
    min_lat = np.min(lat[i - 2:i])
    max_lat = np.max(lat[i - 2:i])
    lon_vals = lon[i - 2:i]
    lat_vals = lat[i - 2:i]
    indices = [i for i in range(param + 1)]
    values = sample_path_batch(1, param + 1)[0]
    # NOTE: Use one brownian bridge at the moment
    for j in range(len(values)):
        j_val = values[j]
        min_val = np.min(values)
        max_val = np.max(values)
#         nval = (lon_vals[0] * (indices[-1] - j) + lon_vals[1] * (j - indices[0])) / (indices[-1] - indices[0])
#         rej_val = min_lon + ((j_val - min_val) * (max_lon - min_lon)) / (max_val - min_val)
        rej_val = ((j_val - min_val) * (max_lon - min_lon)) / (max_val - min_val)
        nval = rej_val + (lon_vals[0] * (indices[-1] - j) + lon_vals[1] * (j - indices[0])) / (indices[-1] - indices[0])
        if j != 0:# and j != len(values):
            lon_nvals.append(nval)
    values = sample_path_batch(1, param + 1)[0]
    for j in range(len(values)):
        j_val = values[j]
        min_val = np.min(values)
        max_val = np.max(values)
        rej_val = ((j_val - min_val) * (max_lat - min_lat)) / (max_val - min_val)
        nval = rej_val + (lat_vals[0] * (indices[-1] - j) + lat_vals[1] * (j - indices[0])) / (indices[-1] - indices[0])
        if j != 0: # and j != len(values):
            lat_nvals.append(nval)

new_latvals = []
new_lonvals = []
for i in range(len(lat_nvals)):
    x, y = utm.to_latlon(lat_nvals[i], lon_nvals[i], 32, "U")
    new_latvals.append(x)
    new_lonvals.append(y)

# coords = zip(lat_nvals, lon_nvals)
coords = zip(new_latvals, new_lonvals)

# # plt.plot(lon_nvals, lat_nvals)
# plt.plot(new_latvals, new_lonvals)
# plt.show()
# len(list(npolyline.coords))
# len(list(lat_nvals))
# for item in coords:
#     print(item)

In [105]:
map_ = setup_map(center)
map_ = plot_point(start_dp, center, color = "red", map_ = map_)
map_ = plot_point(end_dp, center, color = "blue", map_ = map_)
map_ = plot_polyline(
    [start_dp] + [DataPoint(x, y) for x, y in coords] + [end_dp],
    center,
    color = "blue",
    map_ = map_,
)
map_
# map_ = setup_map(center)
# map_ = plot_point(start_dp, center, color = "red", map_ = map_)
# map_ = plot_point(end_dp, center, color = "blue", map_ = map_)
# for point in [DataPoint(x, y) for x, y in coords]:
#     map_ = plot_point(point, center, color = "green", map_ = map_)
# map_ = plot_polyline([DataPoint(x, y) for x, y in coords], center, color = "blue", map_ = map_)
# map_
# plt.scatter([i for i in range(len(lat_nvals))], lat_nvals, s= 5)

# plt.scatter(lon[:1], lat[:1], s = 25, color = "red")
# plt.scatter(lon_nvals, lat_nvals, s = 5)
# plt.scatter([i for i in range(len(lat[:10]))], lat[:10], s = 25, color = "red")
# plt.scatter([i for i in range(len(lon_nvals))], lon_nvals, s = 5)

# plt.scatter([i for i in range(len(lat))], lat, s= 2.5)
# plt.plot(lon_nvals, lat_nvals)
# plt.show()

In [12]:
# map_ = setup_map(center)
# map_ = plot_point(start_dp, center, color = "red", map_ = map_)
# map_ = plot_point(end_dp, center, color = "blue", map_ = map_)
# map_ = plot_polyline(
#     [start_dp] + [DataPoint(x, y) for x, y in polyline] + [end_dp],
#     center,
#     color = "blue",
#     map_ = map_,
# )
# map_